<a href="https://colab.research.google.com/github/Chaitanyaa/ML-CIFAR100/blob/master/CMPE257CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exploring with dowloaded dataset

In [0]:
#1 Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#2. Get the file
#make sure you upload all your data files to your Google drive and change share->Advanced->change->anyone with the link can view
downloaded = drive.CreateFile({'id':'1IUaJmhnauW0GYZKMFRsvWeHpxahosUe4'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train')

In [0]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [0]:
images = unpickle('train')

In [0]:
print(images)

In [0]:
images[b'data'].shape

In [0]:
import pandas as pd
data = pd.DataFrame(images[b'data'])

In [0]:
images.keys()

In [0]:
import numpy as np
np.unique(images[b'coarse_labels'])

## With Cifar-100 dataset in Keras

In [0]:
# install keras first
!pip install dill
!pip install -q keras

In [0]:
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32"

In [0]:
from keras import backend as K
if K.backend()=='tensorflow':
    K.set_image_dim_ordering("th")

In [0]:
#batch_size = 1000
num_classes = 20
'''
epochs_longrun = 500
save_dir = "/work"
res_dir = "/results"
model_name = 'convnet_cifar100'
'''

In [0]:
from keras.datasets import cifar100

In [0]:
from __future__ import print_function
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from keras.layers.pooling import MaxPooling2D
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.utils import to_categorical
from keras.models import load_model

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [0]:
import tensorflow as tf
import numpy as np
from keras.utils import to_categorical
tf.reset_default_graph()
tf.set_random_seed(343)
np.random.seed(343)
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='coarse')
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [0]:
#Combining the training and test data back into one data
x = np.concatenate((x_train,x_test))
y = np.concatenate((y_train,y_test))
print('x shape:', x.shape)
print('y shape:', y.shape)

In [0]:
# Required subset of data in dataframe
x = x.reshape(x.shape[0],3*32*32)
images_df = pd.DataFrame(x, columns = list(range(0,3072)))
temp = pd.DataFrame(y, columns = ['Target'])
images_df = pd.concat([images_df,temp],axis=1)
vehicle_images = images_df[(images_df['Target']==18) | (images_df['Target']==19) ]
vehicle_images.head()

In [0]:
# To view sample data
fig = plt.figure(figsize=(10, 10))
fig.subplots_adjust(hspace=0.4, wspace=0.4)
k=1
for i in x[vehicle_images[vehicle_images['Target']==18].sample(16).index]: # Change Target number here ( 18 = Vehicle 1, 19 = Vehicle 2)
    ax = fig.add_subplot(4, 4, k)
    img = i.reshape(3,32,32).transpose([1, 2, 0])
    fig.set_tight_layout(True) 
    ax=plt.imshow(img)
    k=k+1

In [0]:
x = vehicle_images.drop(['Target'],axis=1)

In [0]:
y = vehicle_images['Target']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.33, random_state=42)

In [0]:
y_train.shape

In [0]:
# Convert class vectors to binary class matrices.
y_train_c = to_categorical(y_train, num_classes)
y_test_c = to_categorical(y_test, num_classes)
#Normalize data
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0

In [0]:
model = Sequential()

n_input = X_train.shape[1]
n_hidden = n_input

model.add(Dense(n_hidden*5, input_dim=n_input, activation='sigmoid'))
model.add(Dense(n_hidden*3, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))#why sigmoid here?

In [0]:
from keras.optimizers import Adam
#compile the model with loss='binary_crossentropy' and optimizer=adam
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['acc'])
model.summary()

In [0]:
#Train the model with X_train, y_train and validation data as X_test, y_test
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=200,batch_size=20,verbose=0);

In [0]:
training_loss = history.history['loss']
test_loss = history.history['val_loss']
# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)
# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

In [0]:
score =model.evaluate(X_test, y_test,  verbose=0)
print('prediction score: %.2f%%' % (score[1]*100)  )

## Prediction with Feedforward Neural Networks


In [0]:
#Let's review how MLPClassifier got trained and do prediction
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(200), activation='logistic')
mlp.fit(X_train, y_train)
score =mlp.score(X_test, y_test)
print('MLP NN score: %.2f%%' %(score*100))